In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
from IPython.display import display, Markdown, Latex

PM = .13
SE = .777
SP = .988

def P_S_n(pm, spec=.5, sens=.5):
    return (spec * (1 - pm)) / (spec * (1 - pm) + (1 - sens) * pm)

def P_M_p(pm, spec=.5, sens=.5):
    return (sens * pm) / (sens * pm + (1 - spec) * (1 - pm))

def latexplot(plt, figlabel, figcaption, showme=False, single=False):

    figname = figlabel + '.png'
    plt.savefig(figname, bbox_inches='tight')

    if showme:
        plt.show()
        return
    plt.close()

    if single:
        strLatex=fr"""
        \begin{{figure}}
        \centering
            \includegraphics[width=0.5\textwidth,height=0.5\textheight,keepaspectratio]{{{figname}}}
            \caption{{{figcaption}}}
            \label{{fig:{figlabel}}}
        \end{{figure}}"""        
    else:
        strLatex=fr"""
        \begin{{figure}}
        \centering
            \includegraphics{{{figname}}}
            \caption{{{figcaption}}}
            \label{{fig:{figlabel}}}
        \end{{figure}}"""
    
    return display(Latex(strLatex)) 

In [2]:
url_ita = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv'
ita = pd.read_csv(url_ita,
                     usecols=['data', 'tamponi', 'casi_testati'],
                     parse_dates=['data'],
                     index_col=['data'],
                     squeeze=True).sort_index()

In [17]:
df = ita.loc["2020-04-19 17:00:00":]
fig, ax = plt.subplots(figsize=(12, 6))

ax.bar(df.index, df["casi_testati"], label="casi testati")
ax.bar(df.index, df["tamponi"]-df["casi_testati"], bottom=df["casi_testati"], label="tamponi eccedenti")
ax.legend(loc="upper left")

ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

ax.set_title("Eccedenza dei casi testati rispetto ai tamponi effettuati")

latexplot(plt, "tamponi", 
         "Casi testati e tamponi effettuati per COVID-19 in Italia.",
         showme=False)

<IPython.core.display.Latex object>

# Introduzione

Recentemente i dati giornalieri pubblicati dal Dipartimento di Protezione Civile sull'epidemia COVID-19 in Italia <cite data-cite="pcm_dpc_2020"></cite> hanno rivelato (figura $\ref{fig:tamponi}$) che la quantità di casi testati totali è attualmente solo il 64% circa del numero di tamponi effettuati (test RT-PCR RNA). Se ne deduce che qualche paziente è stato sottoposto a più di un test. Perché? Quanti test sono necessari o sufficienti per diagnosticare o escludere la patologia?

Questa breve analisi, senza pretendere di essere una trattazione esaustiva e professionale del complesso argomento, vuole introdurre ai concetti base per comprendere i motivi e gli strumenti matematici sottostanti alle ripetizioni di test diagnostici qualitativi.
Per un approfondimento più dettagliato si veda [Approfondimento](https://maxpierini.it/ncov/approfondimento-test.pdf).

# Test diagnostici

I test diagnostici si dividono in tre grandi categorie, in base al tipo di risultato ottenuto <cite data-cite="porta2014dictionary"></cite>:

- Descrittivi
- Quantitativi
- Qualitativi

I risultati dei test **descrittivi** consistono per l'appunto in un'analisi descrittiva (il "referto", dal latino _refero_) del materiale ottenuto (il "reperto", dal latino _repero_). È il caso, ad esempio, di una radiografia o di un esame istologico.

I risultati dei test **quantitativi** si presentano invece come valori numerici il cui significato, sulla base di precise linee guida derivanti da sperimentazioni e pratica clinica, fornisce indicazioni sullo stato di salute. Accade, ad esempio, per un emocromo o la misurazione della pressione sanguigna.

Il risultati dei test **qualitativi** sono invece rappresentati da un responso dicotomico, solitamente caratterizzato dalle qualità "positivo" $\oplus$ o "negativo" $\ominus$, che rivela la presenza o meno di una particolare caratteristica ricercata.

I tamponi naso-faringei utilizzati per la diagnosi di COVID-19 sono proprio di tipo qualitativo: il risultato (ottenuto grazie all'analisi della presenza di specifico RNA virale con metodo PCR) $\oplus$ indica la presenza di infezione da SARS-nCoV-2 virus e quindi di "malattia" $M$ oppure la sua assenza $\ominus$ e dunque di esclusione della situazione patologica $\overline{M}$ (che leggeremo "non $M$") <cite data-cite="padhye2020reconstructed"></cite>.

Tutti i test sono caratterizzati da due importanti parametri che ne descrivono la capacità di identificare i soggetti sani e malati <cite data-cite="porta2014dictionary"></cite>:

- la **sensibilità** $\mathbf{SE}$ è la probabilità di ottenere un test positivo in individui malati, ovvero $P(\oplus|M)$ (che leggeremo "probabilità di $\oplus$ dato $M$")
- la **specificità** $\mathbf{SP}$ è la probabilità di ottenere un test negativo in individui sani, ovvero $P(\ominus|\overline{M})$ (che, similmente, leggeremo "probabilità di $\ominus$ dato non $M$")

Essendo _probabilità_ il loro valore varia da 0 a 1 ovvero da 0% a 100%.

La situazione ideale sarebbe dunque di avere a disposizione un test con $\mathbf{SE}=1$ (100% di test positivi su tutti i malati) e $\mathbf{SP}=1$ (100% di test negativi su tutti i sani). Ma i test diagnostici non sono strumenti perfetti e questi valori raramente sono "tanto vicini 1" da escludere la possibilità di errori non trascurabili.

In particolare, falsi positivi $F_{\oplus}$ sono i soggetti sani che ottengono test positivi e la probabilità di ottenere un falso positivo corrisponde pertanto alla probabilità di avere un test positivo in un individuo sano $P(\oplus|\overline{M})$ che, per le proprietà delle probabilità, corrisponde al complementare della specificità

\begin{equation}\label{eq:falsipositivi}
P(F_{\oplus}) = P(\oplus|\overline{M}) = \overline{P(\ominus|\overline{M})} = 1 - P(\ominus|\overline{M}) =
1 - \mathbf{SP}
\end{equation}

Similmente, falsi negativi $F_{\ominus}$ sono i soggetti malati che ottengono test negativi e la probabilità di ottenere un falso negativo corrisponde pertanto alla probabilità di avere un test negativo in un individuo malato $P(\ominus|M)$ che corrisponde al complementare della sensibilità

\begin{equation}\label{eq:falsinegativi}
P(F_{\ominus}) = P(\ominus|M) = \overline{P(\oplus|M)} = 1 - P(\oplus|M) =
1 - \mathbf{SE}
\end{equation}

Pertanto (vedi figura $\ref{fig:se-sp}$), una bassa sensibilità diminuisce la probabilità di avere test positivi nei malati (veri positivi) e aumenta la probabilità di errore di ottenere test negativi su malati (falsi negativi), mentre una bassa specificità diminuisce la probabilità di test negativi nei sani (veri negativi) e aumenta la probabilità di ottenere test erroneamente positivi nei sani (falsi positivi).

Per ridurre queste probabilità di errore e migliorare $\mathbf{SE}$ o $\mathbf{SP}$ di un test, laddove non ottimali, esistono diverse strategie (consistenti nell'applicazione di più test contemporanei o successivi in base al risultato del precedente) la cui applicazione modifica positivamente un parametro a scapito dell'opposto. L'attenta calibrazione di queste strategie può quindi portare a una riduzione della probabilità degli errori non voluti senza incrementare troppo la probabilità dell'errore opposto <cite data-cite="weinstein2005clinical"></cite>.

\begin{figure}
\centering
    \includegraphics[width=0.5\textwidth,height=0.5\textheight,keepaspectratio]{se-sp}
    \caption{Schema di sensibilità e specificità e rapporti con le probabilità.}
    \label{fig:se-sp}
\end{figure}

# Probabilità di malattia

Un individuo di cui non si abbiano precedenti informazioni, estratto a caso da una popolazione, presenta una probabilità di essere affetto da una specifica patologia $P(M)$ pari alla **prevalenza** della malattia nella popolazione di riferimento. La prevalenza è misurata come numero medio di soggetti malati nella popolazione ovvero come percentuale di soggetti malati e dunque probabilità di estrarre un individuo malato preso a caso dalla popolazione in oggetto.

È il caso, ad esempio, dello _screening_ o dei "test a tappeto": non si hanno ulteriori informazioni sui soggetti se non quella di appartenere ad una determinata popolazione (che può essere vasta come "gli italiani" o più ristretta come "maschi italiani fumatori da più di 10 anni" o "donne in menopausa sopra i 50 anni") ed avere dunque una **probabilità a priori** di malattia pari alla prevalenza della patologia nella popolazione di appartenenza.

Nel caso invece di un paziente che presenti specifici segni, sintomi, storia clinica precedente, fattori di rischio, risultati di precedenti esami ecc, la sua $P(M)$ dipende anche dalla valutazione del suo stato e della sua anamnesi.

Nel corso di un'epidemia, la prevalenza di una particolare patologia infettiva nella popolazione colpita, per ovvi motivi, aumenta: una percentuale nettamente superiore di popolazione è affetta dalla malattia ovvero la probabilità $P(M)$ di essere malati aumenta e diminuisce di conseguenza la probabilità di essere sani $P(\overline{M}) = 1 - P(M)$.

Per COVID-19 in Italia, la prevalenza in fase epidemica è stata stimata al 13% (circa un soggetto su 8) <cite data-cite="ceylan2020estimation"></cite> <cite data-cite="vollmer2020sub"></cite> <cite data-cite="flaxman2020report"></cite>, ovvero un individuo preso a caso dalla popolazione avrebbe una probabilità a priori di essere infetto $P(M)=13\%$ (e la quantità stimata di infetti, anche se asintomatici, nella popolazione italiana sarebbe pari a circa $60'000'000 \cdot 13\% = 7'800'000$ individui!).

Sia in presenza di segni, sintomi, anamnesi caratteristica ecc, che a prescindere da questi, l'applicazione di un test diagnostico (ad esempio, qualitativo) incrementa o diminuisce la probabilità di malattia **a posteriori** dell'esaminato, in particolare:

- $P(M|\oplus)$ la probabilità a posteriori di malattia in seguito a test positivo
- $P(\overline{M}|\ominus)$ la probabilità a posteriori di salute in seguito a test negativo

Il _Teorema di Bayes_ fornisce lo strumento matematico che lega le probabilità a posteriori di malattia dato il risultato di un test a sensibilità e specificità del test utilizzato e alla probabilità a priori di malattia (pari alla semplice prevalenza o alla più complessa valutazione clinica del paziente) <cite data-cite="kruschke2014doing"></cite>.

# Sensibilità e Specificità

Grazie al Teorema di Bayes sappiamo come sensibilità e specificità modifichino la probabilità di malattia a posteriori in seguito al risultato di un test, nota la probabilità di malattia a priori.

Tutti i tre parametri hanno effetto, positivo e negativo, sulla probabilità di malattia a posteriori, ma in particolare:

- la sensibilità $\mathbf{SE}=P(\oplus|M)$ ha maggior effetto sulla probabilità a posteriori di salute in seguito test negativo $P(\overline{M}|\ominus)$ oltre che, come già sappiamo, sul rischio di falsi negativi
- la specificità $\mathbf{SP}=P(\ominus|\overline{M})$ ha maggior effetto sulla probabilità a posteriori di malattia in seguiti a test positivo $P(M|\oplus)$ oltre che, come visto in precedenza, sul rischio di falsi positivi

Questo significa che ad esempio, a parità di specificità, variazioni nella sensibilità di un test produrranno una notevole variazione della probabilità di salute se il test è negativo mentre, a parità di sensibilità, variazioni nella specificità condurranno ad altrettanto notevoli variazioni della probabilità di malattia in seguito a test positivo (vedi figura $\ref{fig:sens-spec}$).

Per approfondimenti si veda [Approfondimento](https://maxpierini.it/ncov/approfondimento-test.pdf).

\begin{figure}
\centering
    \includegraphics{sens-spec}
    \caption{Rapporto tra specificità e sensibilità. Le linee piene indicano $P(M|\oplus)$, le linee tratteggiate $P(M|\ominus).$}
    \label{fig:sens-spec}
\end{figure}

# Tamponi naso-faringei e COVID-19

In [4]:
display(Markdown(fr"""
Recenti studi suggeriscono che i test RT-PCR RNA per COVID-19 (tamponi naso-faringei) 
presentino mediamente sensibilità $\mathbf{{SE}} = {SE} = {SE*100:.2f}\%$ e 
specificità $\mathbf{{SP}}={SP}={SP*100:.2f}\%$ <cite data-cite="padhye2020reconstructed"></cite>.
"""))


Recenti studi suggeriscono che i test RT-PCR RNA per COVID-19 (tamponi naso-faringei) 
presentino mediamente sensibilità $\mathbf{SE} = 0.777 = 77.70\%$ e 
specificità $\mathbf{SP}=0.988=98.80\%$ <cite data-cite="padhye2020reconstructed"></cite>.


Ci troviamo dunque di fronte ad un test piuttosto specifico che comporterà una probabilità relativamente esigua di falsi positivi

In [5]:
display(Markdown(fr"""
$$
P(F_{{\oplus}}) = 1 - P(\ominus|\overline{{M}}) = 1 - \mathbf{{SP}} = 1 - {SP} = {1-SP:.3f} = {(1-SP)*100:.2f}\%
$$
"""))


$$
P(F_{\oplus}) = 1 - P(\ominus|\overline{M}) = 1 - \mathbf{SP} = 1 - 0.988 = 0.012 = 1.20\%
$$


e ad una notevole variazione della probabilità di malattia a posteriori in seguito a risultato positivo.

Ma, al contempo, il test non ha una sensibilità ottimale e il rischio di falsi negativi è decisamente più elevato

In [6]:
display(Markdown(fr"""
$$
P(F_{{\ominus}}) = 1 - P(\oplus|M) = 1 - \mathbf{{SE}} = 1 - {SE} = {1-SE:.3f} = {(1-SE)*100:.2f}\%
$$
"""))


$$
P(F_{\ominus}) = 1 - P(\oplus|M) = 1 - \mathbf{SE} = 1 - 0.777 = 0.223 = 22.30\%
$$


così come si ottiene una minore variazione nella probabilità di salute a posteriori in seguito a risultato negativo.

L'applicazione di una delle strategie a cui abbiamo accennato può però migliorare la scarsa sensibilità senza "degradare" eccessivamente l'accettabile specificità.

# Test ripetuti

A prescindere dall'utilizzo di esami differenti (come radiografie e test sierologici) e dalla rivalutazione clinica delle condizioni del paziente, l'applicazione della strategia nota come **Regola O serial** (the OR rule in serial tests) è volta propriamente ad aumentare la sensibilità di un test (a scapito della specificità):

- il test viene ripetuto $n$ volte solo se precedentemente negativo
- è suﬀiciente che uno solo dei test risulti positivo per confermare la situazione patologica
- solo se tutti gli $n$ test sono negativi è esclusa la malattia.

Data la prevalenza stimata di COVID-19 in Italia e la necessità di dosare attentamente il numero di tamponi effettuati vista la loro scarsità, si rende necessario applicare la regola di ripetizione (vedi figura $\ref{fig:ripetizione-covid}$) solo in particolari casi come soggetti a rischio notevolmente elevato, precedenti ospedalizzati dimessi, pazienti con segni e sintomi rilevanti, ecc che aumentino la probabilità di malattia a posteriori in maniera decisiva rispetto alla semplice prevalenza (vedi figura $\ref{fig:covid}$).

In [7]:
display(Markdown(fr"""
È infatti sufficiente un solo test negativo in un soggetto con probabilità di malattia a priori 
pari alla sola prevalenza $P(M)={PM*100:.2f}$% 
per portare la probabilità di salute a posteriori in seguito a test negativo, al di sopra di una soglia 
accettabile $P(\overline{{M}}|\ominus) = {(P_S_n(PM, SP, SE))*100:.2f}$%
"""))


È infatti sufficiente un solo test negativo in un soggetto con probabilità di malattia a priori 
pari alla sola prevalenza $P(M)=13.00$% 
per portare la probabilità di salute a posteriori in seguito a test negativo, al di sopra di una soglia 
accettabile $P(\overline{M}|\ominus) = 96.74$%


In [8]:
display(Markdown(fr"""
Al contempo, è sufficiente un solo test positivo in un soggetto con probabilità di malattia a priori 
pari alla sola prevalenza 
per aumentare notevolmente la probabilità di malattia a posteriori in seguito a test positivo 
 $P(M|\oplus) = {(P_M_p(PM, SP, SE))*100:.2f}$%
"""))


Al contempo, è sufficiente un solo test positivo in un soggetto con probabilità di malattia a priori 
pari alla sola prevalenza 
per aumentare notevolmente la probabilità di malattia a posteriori in seguito a test positivo 
 $P(M|\oplus) = 90.63$%


In [9]:
new_PM = (1 - P_S_n(P_M_p(PM, SP, SE), SP, SE))
display(Markdown(fr"""
Assumendo per semplicità quest'ultima come nuova probabilità a priori di malattia in un soggetto 
precedentemente diagnosticato come malato, ad elevato rischio o con segni e sintomi caratteristici, 
effettuando un test per verificare che sia sano o guarito e presumendo che risulti negativo, 
la sua probabilità di salute a posteriori non è più accettabile, avrebbe infatti 
$P(\overline{{M}}|\ominus) = {P_S_n(P_M_p(PM, SP, SE), SP, SE)*100:.2f}\%$ e quindi ancora una
probabilità di essere malato notevolmente elevata 
$P(M|\ominus) = 1 - P(\overline{{M}}|\ominus) = {new_PM*100:.2f}\%$
"""))


Assumendo per semplicità quest'ultima come nuova probabilità a priori di malattia in un soggetto 
precedentemente diagnosticato come malato, ad elevato rischio o con segni e sintomi caratteristici, 
effettuando un test per verificare che sia sano o guarito e presumendo che risulti negativo, 
la sua probabilità di salute a posteriori non è più accettabile, avrebbe infatti 
$P(\overline{M}|\ominus) = 31.41\%$ e quindi ancora una
probabilità di essere malato notevolmente elevata 
$P(M|\ominus) = 1 - P(\overline{M}|\ominus) = 68.59\%$


In [10]:
display(Markdown(fr"""
A prescindere quindi (come si diceva) dalla valutazione clinica e/o da altri esami effettuati, 
dato che il test è risultato negativo, si può procedere con la **Regola O** sottoponendolo 
ad un nuovo tampone. Come accennato però, le strategie di ripetizione modificano sia sensibilità che 
specificità e (applicando le formule corrispondenti) otterremo un test con

$$
\mathbf{{SE}}_2 = 1 - (1 - \mathbf{{SE}})^2 = 1 - (1 - {SE})^2 = {1-(1-SE)**2:.4f} = {(1-(1-SE)**2)*100:.2f}\%
$$

$$
\mathbf{{SP}}_2 = \mathbf{{SP}}^2 = {SP}^2 = {SP**2:.4f} = {(SP**2)*100:.2f}\%
$$
"""))


A prescindere quindi (come si diceva) dalla valutazione clinica e/o da altri esami effettuati, 
dato che il test è risultato negativo, si può procedere con la **Regola O** sottoponendolo 
ad un nuovo tampone. Come accennato però, le strategie di ripetizione modificano sia sensibilità che 
specificità e (applicando le formule corrispondenti) otterremo un test con

$$
\mathbf{SE}_2 = 1 - (1 - \mathbf{SE})^2 = 1 - (1 - 0.777)^2 = 0.9503 = 95.03\%
$$

$$
\mathbf{SP}_2 = \mathbf{SP}^2 = 0.988^2 = 0.9761 = 97.61\%
$$


\begin{figure}
\centering
    \includegraphics[width=0.5\textwidth,height=0.5\textheight,keepaspectratio]{covid}
    \caption{Probabilità di malattia COVID-19 a posteriori per test RT-PCR SARS-CoV-2 RNA con ripetizione seriale dei test Regola $\mathbf{O}$}
    \label{fig:covid}
\end{figure}

In [11]:
new_PM2 = (1 - P_S_n(new_PM, SP**2, 1-(1-SE)**2))
display(Markdown(fr"""
Abbiamo pertanto incrementato la sensibilità del {(1-(1-SE)**2-SE)*100:.2f}% a scapito 
di una diminuzione della specificità pari al {(SP**2-SP)*100:.2f}%.

Sulla base di questi nuovi parametri effettuiamo un nuovo test assumendo come probabilità 
di malattia a priori il risultato ottenuto dal test precedente $P(M)={new_PM*100:.2f}$%

$$
P(M|\oplus) = {(P_M_p(new_PM, SP**2, 1-(1-SE)**2))*100:.2f}\%
$$

$$
P(\overline{{M}}|\ominus) = {(1 - new_PM2)*100:.2f}\%
$$
"""))


Abbiamo pertanto incrementato la sensibilità del 17.33% a scapito 
di una diminuzione della specificità pari al -1.19%.

Sulla base di questi nuovi parametri effettuiamo un nuovo test assumendo come probabilità 
di malattia a priori il risultato ottenuto dal test precedente $P(M)=68.59$%

$$
P(M|\oplus) = 98.86\%
$$

$$
P(\overline{M}|\ominus) = 89.99\%
$$


In [12]:
display(Markdown(fr"""
Notiamo che effettivamente un risultato positivo confermerebbe la diagnosi di malattia 
(e il falso negativo del test precedente) ma se il test risultasse negativo, nonostante la 
probabilità di salute sia molto più elevata rispetto al test precedente, otterremmo comunque una 
probabilità di malattia a posteriori non ottimale per escludere la patologia 
$P(M|\ominus) = 1 - P(\overline{{M}}|\ominus) = 
{(new_PM2)*100:.2f}$% dato anche il rischio di contagio in corso di epidemia.
"""))


Notiamo che effettivamente un risultato positivo confermerebbe la diagnosi di malattia 
(e il falso negativo del test precedente) ma se il test risultasse negativo, nonostante la 
probabilità di salute sia molto più elevata rispetto al test precedente, otterremmo comunque una 
probabilità di malattia a posteriori non ottimale per escludere la patologia 
$P(M|\ominus) = 1 - P(\overline{M}|\ominus) = 
10.01$% dato anche il rischio di contagio in corso di epidemia.


In [13]:
display(Markdown(fr"""
Perciò, se il test fosse risultato negativo, applicando nuovamente la **Regola O** potremmo sottoporre il soggetto 
ad un nuovo tampone, sapendo però che influirà ancora su sensibilità e 
specificità 

$$
\mathbf{{SE}}_3 = 1 - (1 - \mathbf{{SE}})^3 = 1 - (1 - {SE})^3 = {1-(1-SE)**3:.4f} = {(1-(1-SE)**3)*100:.2f}\%
$$

$$
\mathbf{{SP}}_3 = \mathbf{{SP}}^3 = {SP}^3 = {SP**3:.4f} = {(SP**3)*100:.2f}\%
$$
"""))


Perciò, se il test fosse risultato negativo, applicando nuovamente la **Regola O** potremmo sottoporre il soggetto 
ad un nuovo tampone, sapendo però che influirà ancora su sensibilità e 
specificità 

$$
\mathbf{SE}_3 = 1 - (1 - \mathbf{SE})^3 = 1 - (1 - 0.777)^3 = 0.9889 = 98.89\%
$$

$$
\mathbf{SP}_3 = \mathbf{SP}^3 = 0.988^3 = 0.9644 = 96.44\%
$$


In [14]:
new_PM3 = 1 - (P_S_n(new_PM2, SP**3, 1-(1-SE)**3))
display(Markdown(fr"""
Otterremo dunque un incremento totale della sensibilità del {(1-(1-SE)**3-SE)*100:.2f}% a scapito 
di una diminuzione complessiva della specificità del {(SP**3-SP)*100:.2f}%.

Effettuiamo il terzo test coi nuovi parametri e assumendo come probabilità 
di malattia a priori il risultato ottenuto dal secondo test negativo $P(M)={new_PM2*100:.2f}$%

$$
P(M|\oplus) = {(P_M_p(new_PM2, SP**3, 1-(1-SE)**3))*100:.2f}\%
$$

$$
P(\overline{{M}}|\ominus) = {(1-new_PM3)*100:.2f}\%
$$
"""))


Otterremo dunque un incremento totale della sensibilità del 21.19% a scapito 
di una diminuzione complessiva della specificità del -2.36%.

Effettuiamo il terzo test coi nuovi parametri e assumendo come probabilità 
di malattia a priori il risultato ottenuto dal secondo test negativo $P(M)=10.01$%

$$
P(M|\oplus) = 75.57\%
$$

$$
P(\overline{M}|\ominus) = 99.87\%
$$


In [15]:
display(Markdown(fr"""
Siamo giunti pertanto, grazie alle regole di ripetizione dei test diagnostici, ad ottenere una probabilità 
di malattia a posteriori del terzo tampone negativo pari ad un esiguo $P(M|\ominus)={new_PM3*100:.2f}$% ma 
al contempo avremmo una probabilità di malattia in caso di test positivo sufficiente a ritenere il paziente 
non guarito o comunque molto probabilmente infetto e attuare le necessarie azioni successive (isolamento, 
ospedalizzazione, altri esami, terapie, ecc) a seconda delle condizioni generali del paziente, dei 
suoi fattori di rischio, ecc <cite data-cite="centers2020interim"></cite> 
<cite data-cite="bai2020presumed"></cite> <cite data-cite="national2020coronavirus"></cite>.
"""
))


Siamo giunti pertanto, grazie alle regole di ripetizione dei test diagnostici, ad ottenere una probabilità 
di malattia a posteriori del terzo tampone negativo pari ad un esiguo $P(M|\ominus)=0.13$% ma 
al contempo avremmo una probabilità di malattia in caso di test positivo sufficiente a ritenere il paziente 
non guarito o comunque molto probabilmente infetto e attuare le necessarie azioni successive (isolamento, 
ospedalizzazione, altri esami, terapie, ecc) a seconda delle condizioni generali del paziente, dei 
suoi fattori di rischio, ecc <cite data-cite="centers2020interim"></cite> 
<cite data-cite="bai2020presumed"></cite> <cite data-cite="national2020coronavirus"></cite>.


\begin{figure}
\centering
    \includegraphics{ripetizione-covid}
    \caption{Effetto delle regole di ripetizione sui test RT-PCR per COVID-19}
    \label{fig:ripetizione-covid}
\end{figure}

# Conclusioni

Grazie alle proprietà matematiche di sensibilità, specificità, probabilità di malattia a priori e a posteriori e regole di ripetizione dei test diagnostici qualitativi, si può giungere al miglioramento della non ottimale sensibilità dei tamponi naso-faringei utilizzati nella diagnosi di COVID-19, diminuendo il rischio di falsi negativi e ottenendo un'accettabile probabilità di salute in seguito a test negativo dopo il terzo tampone ripetuto con Regola **O** (solo se negativo) senza rischiare un'eccessiva diminuzione della buona specificità iniziale e mantenendo dunque una probabilità accettabile di falsi positivi e di malattia a seguito di test positivo. L'analisi spiega pertanto, vista l'importanza dei falsi negativi per il contenimento di un'evento epidemico, la discrepanza tra tamponi effettuati e casi totali testati nel corso dell'attuale epidemia di COVID-19 in Italia.